In [1]:
from PyPDF2 import PdfReader
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain_community.callbacks.manager import get_openai_callback
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from typing import List, Tuple

def extract_text_with_page_numbers(pdf) -> Tuple[str, List[int]]:
    """
    从PDF中提取文本并记录每行文本对应的页码
    
    参数:
        pdf: PDF文件对象
    
    返回:
        text: 提取的文本内容
        page_numbers: 每行文本对应的页码列表
    """
    text = ""
    page_numbers = []

    for page_number, page in enumerate(pdf.pages, start=1):
        extracted_text = page.extract_text()
        if extracted_text:
            text += extracted_text
            page_numbers.extend([page_number] * len(extracted_text.split("\n")))
        else:
            Logger.warning(f"No text found on page {page_number}.")

    return text, page_numbers

def process_text_with_splitter(text: str, page_numbers: List[int]) -> FAISS:
    """
    处理文本并创建向量存储
    
    参数:
        text: 提取的文本内容
        page_numbers: 每行文本对应的页码列表
    
    返回:
        knowledgeBase: 基于FAISS的向量存储对象
    """
    # 创建文本分割器，用于将长文本分割成小块
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " ", ""],
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    # 分割文本
    chunks = text_splitter.split_text(text)
    #Logger.debug(f"Text split into {len(chunks)} chunks.")
    print(f"文本被分割成 {len(chunks)} 个块。")
        
    # 创建嵌入模型
    embeddings = OpenAIEmbeddings()
    # 从文本块创建知识库
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    #Logger.info("Knowledge base created from text chunks.")
    print("已从文本块创建知识库。")
    
    # 存储每个文本块对应的页码信息
    knowledgeBase.page_info = {chunk: page_numbers[i] for i, chunk in enumerate(chunks)}

    return knowledgeBase

# 读取PDF文件
pdf_reader = PdfReader('./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf')
# 提取文本和页码信息
text, page_numbers = extract_text_with_page_numbers(pdf_reader)
text

'百度文库  - 好好学习，天天向上  \n-1 上海浦东发展银行西安分行  \n个金客户经理管理考核暂行办法  \n \n \n第一章  总   则 \n第一条   为保证我分行个金客户经理制的顺利实施，有效调动个\n金客户经理的积极性，促进个金业务快速、稳定地发展，根据总行《上\n海浦东发展银行个人金融营销体系建设方案（试行）》要求，特制定\n《上海浦东发展银行西安分行个金客户经理管理考核暂行办法（试\n行）》（以下简称本办法）。  \n第二条   个金客户经理系指各支行（营业部）从事个人金融产品\n营销与市场开拓，为我行个人客户提供综合银行服务的我行市场人\n员。 \n第三条   考核内容分为二大类， 即个人业绩考核、 工作质量考核。\n个人业绩包括个人资产业务、负债业务、卡业务。工作质量指个人业\n务的资产质量。  \n第四条   为规范激励规则，客户经理的技术职务和薪资实行每年\n考核浮动。客户经理的奖金实行每季度考核浮动，即客户经理按其考\n核内容得分与行员等级结合，享受对应的行员等级待遇。  \n 百度文库  - 好好学习，天天向上  \n-2 第二章  职位设置与职责  \n第五条   个金客户经理职位设置为：客户经理助理、客户经理、\n高级客户经理、资深客户经理。  \n第六条   个金客户经理的基本职责：  \n（一）   客户开发。研究客户信息、联系与选择客户、与客户建\n立相互依存、相互支持的业务往来关系，扩大业务资源，创造良好业\n绩； \n（二）业务创新与产品营销。把握市场竞争变化方向，开展市场\n与客户需 求的调研，对业务产品及服务进行创新；设计客户需求的产\n品组合、制订和实施市场营销方案；  \n（三）客户服务。负责我行各类表内外授信业务及中间业务的受\n理和运作，进行综合性、整体性的客户服务；  \n（四）防范风险，提高收益。提升风险防范意识及能力，提高经\n营产品质量；  \n（五）培养人材。在提高自身综合素质的同时，发扬团队精神，\n培养后备业务骨干。  \n 百度文库  - 好好学习，天天向上  \n-3 第三章  基础素质要求  \n第七条   个金客户经理准入条件：  \n（一）工作经历：须具备大专以上学历，至少二年以上银行工作\n经验。  \n（二）工作能力：熟悉我行的各项业务，了解市场情况，熟悉各\n类客户的金融需求

In [2]:
print(f"提取的文本长度: {len(text)} 个字符。")
    
# 处理文本并创建知识库
knowledgeBase = process_text_with_splitter(text, page_numbers)
knowledgeBase

提取的文本长度: 3881 个字符。
文本被分割成 5 个块。
已从文本块创建知识库。


In [ ]:
# 设置查询问题
query = "客户经理被投诉了，投诉一次扣多少分"
query = "客户经理每年评聘申报时间是怎样的？"
if query:
    # 执行相似度搜索，找到与查询相关的文档
    docs = knowledgeBase.similarity_search(query)

    # 初始化OpenAI语言模型
    llm = OpenAI(
        api_key = 'sk-xx',
        base_url = 'https://openai.itit.cc/v1'
    )
    # 加载问答链
    chain = load_qa_chain(llm, chain_type="stuff")

    # 准备输入数据
    input_data = {"input_documents": docs, "question": query}

    # 使用回调函数跟踪API调用成本
    with get_openai_callback() as cost:
        # 执行问答链
        response = chain.invoke(input=input_data)
        print(f"查询已处理。成本: {cost}")
        print(response["output_text"])
        print("来源:")

    # 记录唯一的页码
    unique_pages = set()

    # 显示每个文档块的来源页码
    for doc in docs:
        text_content = getattr(doc, "page_content", "")
        source_page = knowledgeBase.page_info.get(
            text_content.strip(), "未知"
        )

        if source_page not in unique_pages:
            unique_pages.add(source_page)
            print(f"文本块页码: {source_page}")

C:\Users\cheny\AppData\Local\Temp\ipykernel_50024\2938395353.py:14: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


查询已处理。成本: Tokens Used: 3866
	Prompt Tokens: 3836
	Completion Tokens: 30
Successful Requests: 1
Total Cost (USD): $0.005814
 The annual declaration time for the evaluation and appointment of client managers is in January, and the qualification exam is held in February by the personal banking department.
来源:
文本块页码: 1
